# GenParse: Lark interface

In [1]:
import lark

from genparse.util import LarkStuff
from arsenal import Integerizer
from collections import Counter

## Using lark as a front end

In [2]:
grammar2 = r"""
WS: /[ \t\f\r\n]/
STAR: "*"
NUMBER: /\d+/

start: WS "SELECT" WS select_expr WS "FROM" WS from_expr
  [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
EOS: "</s>"
select_expr: STAR | select_list
bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")" 
bool_expr: var "=" value | var ">" value | var "<" value
from_expr: "data"
orderby_expr: var_list WS "ASC" | var_list WS "DESC"
select_list: select_var ("," WS select_var)*
var_list: var ("," WS var)*
select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
value: NUMBER | "red" | "blue" | "white" | "black" | "latino" | "republican" | "democrat" | "male" | "female"

"""

In [3]:
grammar1 = """
start: query_expr EOS

EOS: "</s>"

query_expr: select [ "ORDER" "BY" (order_by_expr ",")*  order_by_expr] [ "LIMIT" integer_ ] 

select: "SELECT" [(select_expr ",")*] select_expr "FROM" "data" [ "WHERE" bool_expression ] [ "GROUP" "BY" [(expression ",")*] expression ]

select_expr.0: expression_math [ [ "AS" ] alias ] -> select_expression

?expression_math: expression_product
               | expression_math PLUS expression_product -> expression_add
               | expression_math "-" expression_product -> expression_sub
               | AGGREGATION expression_math /\)/ -> sql_aggregation

?expression: (name | STAR) -> column_name
            | literal

?expression_product: expression_parens
                  | expression_product STAR expression_parens
                  | expression_product "/" expression_parens 

?expression_parens: expression
                  | /\(/ expression_parens STAR expression /\)/ 
                  | /\(/  expression_parens "/" expression /\)/ 
                  | /\(/  expression_parens PLUS expression /\)/
                  | /\(/  expression_parens "-" expression /\)/

bool_expression: bool_parentheses
                 | bool_expression "AND" bool_parentheses 
                 | bool_expression "OR" bool_parentheses
bool_parentheses: comparison_type
                 | /\(/   bool_expression "AND" comparison_type /\)/
                 | /\(/  bool_expression "OR" comparison_type /\)/
comparison_type: equals | not_equals | greater_than | less_than | greater_than_or_equal
| less_than_or_equal | is_null | is_not_null
equals: expression_math "=" expression_math
not_equals: expression_math ("<>" | "!=") expression_math
greater_than: expression_math ">" expression_math
less_than: expression_math "<" expression_math
greater_than_or_equal: expression_math ">=" expression_math
less_than_or_equal: expression_math "<=" expression_math
is_null: expression_math "is" "null"
is_not_null: expression_math "is" "not" "null"

alias: /[A-Za-z]+/
name: /[A-Za-z]+/
PLUS: /\+/

order_by_expr: expression_math ["ASC"] -> order_asc
        | expression_math "DESC" -> order_desc

AGGREGATION.8: ("sum(" | "avg(" | "min(" | "max(" | "count(" "distinct" | "count(")
STAR: /\*/
integer_: /[1-9][0-9]*/
?literal: boolean -> bool
       | integer_ -> number
       | ESCAPED_STRING -> string

boolean: "true" -> true
       | "false" -> false

%import common.WS
%ignore WS
%import common.ESCAPED_STRING
    
"""

The following code is adapted from partenon.

In [4]:
raw_grammar = grammar1

lark_stuff = LarkStuff(raw_grammar)

In [5]:
intern = Integerizer()   # rename nonterminals to integers
g = lark_stuff.convert()
g = g.rename(intern)
assert g.in_cnf()    # lark returns a grammar in CNF
#g = g.cnf

In [6]:
g

1.0: 1 → 2 3
0.5: 4 → 4 5
1.0: 6 → 7 8
0.07692307692307693: 9 → 2 10
1.0: 11 → 12 13
1.0: 14 → 15 16
1.0: 7 → AND
0.06666666666666667: 17 → 2 18
1.0: 19 → 20 21
1.0: 22 → 23 24
1.0: 25 → 2 26
1.0: 27 → 12 28
1.0: 29 → 23 30
0.16666666666666666: 15 → __ANON_7
1.0: 31 → 32 15
0.09090909090909091: 33 → 2 34
0.1: 35 → TRUE
0.1111111111111111: 36 → 2 26
0.058823529411764705: 37 → 38 11
1.0: 39 → 17 40
0.25: 41 → TRUE
1.0: 42 → 43 14
0.1: 35 → ESCAPED_STRING
1.0: 44 → 45 22
0.08333333333333333: 21 → 46 47
0.058823529411764705: 37 → 38 27
0.08333333333333333: 21 → STAR
1.0: 40 → 45 48
0.058823529411764705: 49 → 2 50
0.06666666666666667: 2 → 46 51
0.1111111111111111: 36 → 2 34
0.08333333333333333: 21 → FALSE
0.08333333333333333: 52 → 38 53
1.0: 54 → 45 23
1.0: 26 → 55 2
0.06666666666666667: 17 → 2 19
1.0: 56 → 45 57
0.08333333333333333: 21 → __ANON_6
0.1: 35 → 46 58
1.0: 59 → MINUS
0.06666666666666667: 17 → 21 60
0.06666666666666667: 2 → 21 61
1.0: 62 → DESC
0.058823529411764705: 37 → 52 63
1.0: 43 → STAR
1.0: 64 → 45 65
1.0: 66 → 23 67
1.0: 68 → 32 15
0.06666666666666667: 17 → 2 69
1.0: 70 → 45 71
0.09090909090909091: 33 → 2 72
1.0: 73 → 2 74
1.0: 51 → 35 42
1.0: 75 → 76 77
1.0: 78 → 76 15
1.0: 55 → __ANON_5
0.1: 35 → __ANON_7
1.0: 79 → 23 80
0.5: 32 → 15 81
1.0: 82 → 83 84
1.0: 85 → 12 86
1.0: 87 → 17 88
1.0: 47 → 35 89
0.058823529411764705: 37 → 38 90
0.07692307692307693: 9 → 2 74
0.06666666666666667: 2 → TRUE
0.058823529411764705: 49 → STAR
1.0: 91 → 15 81
0.058823529411764705: 37 → 38 92
1.0: 71 → 23 93
1.0: 94 → LESSTHAN
0.1111111111111111: 36 → 2 72
0.5: 95 → TRUE
0.058823529411764705: 49 → 96 97
1.0: 98 → 17 56
1.0: 99 → 100 101
0.058823529411764705: 49 → ESCAPED_STRING
1.0: 102 → 103 33
0.07692307692307693: 9 → 2 104
1.0: 105 → 106 107
1.0: 108 → 9 109
1.0: 46 → __ANON_1
1.0: 34 → 110 2
1.0: 90 → 17 111
0.16666666666666666: 15 → FALSE
0.5: 112 → 2 10
1.0: 45 → FROM
0.06666666666666667: 2 → 2 19
0.16666666666666666: 15 → __ANON_6
0.06666666666666667: 17 → 2 50
1.0: 107 → __ANON_7
1.0: 83 → NOT
1.0: 28 → 17 70
1.0: 113 → __ANON_6
1.0: 111 → 45 114
0.06666666666666667: 2 → 46 58
1.0: 92 → 17 64
0.058823529411764705: 37 → 38 87
1.0: 115 → 76 49
0.058823529411764705: 49 → 46 58
0.08333333333333333: 21 → 46 116
1.0: 20 → PLUS
1.0: 84 → NULL
0.058823529411764705: 49 → 2 62
0.09090909090909091: 33 → 2 26
0.07692307692307693: 9 → 2 3
0.06666666666666667: 17 → 21 61
1.0: 24 → 100 117
1.0: 118 → MORETHAN
1.0: 10 → 119 2
0.07692307692307693: 9 → 2 120
0.058823529411764705: 49 → __ANON_7
1.0: 101 → 76 15
0.058823529411764705: 37 → 38 121
0.058823529411764705: 37 → 52 122
1.0: 109 → 100 123
0.058823529411764705: 37 → 38 124
0.058823529411764705: 37 → 52 125
0.1: 35 → 46 47
1.0: 126 → 12 127
0.1: 35 → STAR
0.06666666666666667: 17 → STAR
0.058823529411764705: 37 → 38 128
1.0: 3 → 129 82
0.08333333333333333: 21 → 46 51
1.0: 130 → 45 23
1.0: 104 → 131 2
0.1: 35 → FALSE
1.0: 72 → 118 2
1.0: 124 → 12 39
1.0: 89 → 20 132
0.25: 41 → ESCAPED_STRING
0.1: 35 → __ANON_6
1.0: 133 → 49 134
0.06666666666666667: 17 → ESCAPED_STRING
1.0: 135 → 9 6
0.08333333333333333: 52 → 38 126
0.09090909090909091: 33 → 46 135
1.0: 136 → 76 15
0.08333333333333333: 52 → 38 85
0.06666666666666667: 2 → 2 50
1.0: 137 → ASC
1.0: 38 → SELECT
0.09090909090909091: 33 → 2 138
1.0: 129 → IS
0.058823529411764705: 37 → 52 139
1.0: 140 → 100 78
1.0: 141 → 45 142
0.5: 12 → 17 81
1.0: 143 → 32 15
0.06666666666666667: 17 → 46 58
1.0: 13 → 17 144
0.07692307692307693: 9 → 2 72
0.058823529411764705: 37 → 38 98
0.07692307692307693: 9 → 46 145
0.1111111111111111: 36 → 2 138
0.25: 41 → __ANON_7
0.06666666666666667: 17 → __ANON_7
0.06666666666666667: 2 → 46 47
1.0: 146 → SLASH
1.0: 60 → 146 35
0.06666666666666667: 2 → STAR
0.058823529411764705: 49 → 46 47
1.0: 5 → 49 81
1.0: 147 → 100 148
1.0: 0 → 37 149
0.06666666666666667: 2 → 96 97
1.0: 76 → BY
0.08333333333333333: 21 → 21 60
0.06666666666666667: 2 → ESCAPED_STRING
0.058823529411764705: 49 → FALSE
0.058823529411764705: 49 → __ANON_6
1.0: 10

In [7]:
len(g.rules), len(g.V), len(g.N)

(318, 38, 184)

In [8]:
sorted(g.cnf.V)

['AGGREGATION',
 'AND',
 'AS',
 'ASC',
 'BY',
 'COMMA',
 'DATA',
 'DESC',
 'EOS',
 'EQUAL',
 'ESCAPED_STRING',
 'FALSE',
 'FROM',
 'GROUP',
 'IS',
 'LESSTHAN',
 'LIMIT',
 'MINUS',
 'MORETHAN',
 'NOT',
 'NULL',
 'OR',
 'ORDER',
 'PLUS',
 'SELECT',
 'SLASH',
 'STAR',
 'TRUE',
 'WHERE',
 'WS',
 '__ANON_0',
 '__ANON_1',
 '__ANON_2',
 '__ANON_3',
 '__ANON_4',
 '__ANON_5',
 '__ANON_6',
 '__ANON_7']

In [9]:
#from newton.linking import LinkAnalysis
#f = Integerizer()
#links = LinkAnalysis(g.rename(f))
#links.dfs

In [10]:
g.language(6)

Counter({('SELECT', 'STAR', 'FROM', 'DATA', 'EOS'): 0.00392156862745098,
         ('SELECT',
          'ESCAPED_STRING',
          'FROM',
          'DATA',
          'EOS'): 0.00392156862745098,
         ('SELECT', '__ANON_7', 'FROM', 'DATA', 'EOS'): 0.00392156862745098,
         ('SELECT', '__ANON_6', 'FROM', 'DATA', 'EOS'): 0.00392156862745098,
         ('SELECT',
          'STAR',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.000326797385620915,
         ('SELECT',
          'ESCAPED_STRING',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.000326797385620915,
         ('SELECT',
          '__ANON_7',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.000326797385620915,
         ('SELECT',
          '__ANON_6',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.000326797385620915,
         ('S

## Tokenization

We can extract lark's tokenizer in a format that we can build on.  We will even make a DIY tokenizer based on Python's `re` library.

| Terminology  |         |
|--------------|---------|
| tokenization | lexing  |
| tokenizers   | lexers  |
| tokens       | lexemes |



In [11]:
sorted(lark_stuff.all_terminals, key=lambda t: -t.priority)

[TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()'),
 TerminalDef('ESCAPED_STRING', '".*?(?<!\\\\)(\\\\\\\\)*?"'),
 TerminalDef('WS', '(?:[ \t\x0c\r\n])+'),
 TerminalDef('EOS', '</s>'),
 TerminalDef('PLUS', '\\+'),
 TerminalDef('STAR', '\\*'),
 TerminalDef('COMMA', ','),
 TerminalDef('ORDER', 'ORDER'),
 TerminalDef('BY', 'BY'),
 TerminalDef('LIMIT', 'LIMIT'),
 TerminalDef('WHERE', 'WHERE'),
 TerminalDef('GROUP', 'GROUP'),
 TerminalDef('SELECT', 'SELECT'),
 TerminalDef('FROM', 'FROM'),
 TerminalDef('DATA', 'data'),
 TerminalDef('AS', 'AS'),
 TerminalDef('MINUS', '\\-'),
 TerminalDef('__ANON_0', '\\)'),
 TerminalDef('SLASH', '/'),
 TerminalDef('__ANON_1', '\\('),
 TerminalDef('AND', 'AND'),
 TerminalDef('OR', 'OR'),
 TerminalDef('EQUAL', '='),
 TerminalDef('__ANON_2', '<>'),
 TerminalDef('__ANON_3', '!='),
 TerminalDef('MORETHAN', '>'),
 TerminalDef('LESSTHAN', '<'),
 TerminalDef('__ANON_4', '>='),
 TerminalDef('__ANON_5', '<='),
 TerminalDef('IS', 'i

### DIY tokenizer

In [12]:
text = "12 + 24 - 36 * 48 / 60 SELECT table.name AS thing WHERE table.potato IS NOT 'banana'"

for x, y in lark_stuff.simple_tokenizer(text):
    print(f'{x:15s} -> {y!r}')

__ANON_7        -> '12'
PLUS            -> '+'
__ANON_7        -> '24'
MINUS           -> '-'
__ANON_7        -> '36'
STAR            -> '*'
__ANON_7        -> '48'
SLASH           -> '/'
__ANON_7        -> '60'
SELECT          -> 'SELECT'
__ANON_6        -> 'table'
__ANON_6        -> 'name'
AS              -> 'AS'
__ANON_6        -> 'thing'
WHERE           -> 'WHERE'
__ANON_6        -> 'table'
__ANON_6        -> 'potato'
__ANON_6        -> 'IS'
__ANON_6        -> 'NOT'
__ANON_6        -> 'banana'


### Parsing tokenized input

In [13]:
text = 'SELECT name FROM data </s>'

In [14]:
tokens = list(lark_stuff.lex(text))
tokens

[Token('SELECT', 'SELECT'),
 Token('__ANON_6', 'name'),
 Token('FROM', 'FROM'),
 Token('DATA', 'data'),
 Token('EOS', '</s>')]

Call the lark parser on the text:

In [15]:
lark_stuff.instance.parse(text)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'query_expr'), [Tree(Token('RULE', 'select'), [Tree('select_expression', [Tree('column_name', [Tree(Token('RULE', 'name'), [Token('__ANON_6', 'name')])]), None]), None, None]), None, None]), Token('EOS', '</s>')])

We can call the lark parser on these tokens:

In [16]:
lark_stuff.parser.parse(tokens, 'start')

Tree(NonTerminal(Token('RULE', 'start')), [Tree(NonTerminal(Token('RULE', 'query_expr')), [Tree(NonTerminal(Token('RULE', 'select')), [Token('SELECT', 'SELECT'), Tree(NonTerminal(Token('RULE', 'select_expr')), [Tree(NonTerminal(Token('RULE', 'expression_math')), [Tree(NonTerminal(Token('RULE', 'expression_product')), [Tree(NonTerminal(Token('RULE', 'expression_parens')), [Tree(NonTerminal(Token('RULE', 'expression')), [Tree(NonTerminal(Token('RULE', 'name')), [Token('__ANON_6', 'name')])])])])])]), Token('FROM', 'FROM'), Token('DATA', 'data')])]), Token('EOS', '</s>')])

We can call our parser on this text to get its total weight

In [17]:
g([t.type for t in tokens])

0.00392156862745098

### Greenery Tokenizer

In [18]:
import greenery

def make_greenery_fsms(
    regex_list, 
    ignore = ("\s*",), 
    chars = tuple(chr(i) for i in range(128))
):
    match ignore:
        case []:
            ignore_regex = ""
        case [ignore]:
            ignore_regex = ignore
        case _:
            raise ValueError("ignore must be a list of length at most 1")

    patterns = []
    for regex in regex_list:
        # greenery does not escape spaces
        regex = regex.replace("\\ ", " ")
        patterns.append(greenery.parse(regex + ignore_regex))

    return [pattern.to_fsm() for pattern in patterns]

In [19]:
fsms = make_greenery_fsms([t for t in tokens])

In [20]:
fsms[0]

Fsm(alphabet=frozenset({Charclass((('E', 'E'),)), Charclass((('T', 'T'),)), Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))), ~Charclass((('\t', 'T'),)), Charclass((('L', 'L'),)), Charclass((('\t', '\r'), (' ', ' '))), Charclass((('\x0e', '\x1f'),)), Charclass((('S', 'S'),)), Charclass((('C', 'C'),))}), states=frozenset({0, 1, 2, 3, 4, 5, 6, 7}), initial=0, finals=frozenset({7}), map={0: {Charclass((('\t', '\r'), (' ', ' '))): 1, Charclass((('\x0e', '\x1f'),)): 1, Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))): 1, Charclass((('C', 'C'),)): 1, Charclass((('E', 'E'),)): 1, Charclass((('L', 'L'),)): 1, Charclass((('S', 'S'),)): 2, Charclass((('T', 'T'),)): 1, ~Charclass((('\t', 'T'),)): 1}, 1: {Charclass((('\t', '\r'), (' ', ' '))): 1, Charclass((('\x0e', '\x1f'),)): 1, Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))): 1, Charclass((('C', 'C'),)): 1, Charclass((('E', 'E'),)): 1, Charclass((('L', 'L'),)): 1, Charclass((('S', 'S'),)): 1, Charclass((('T', '

The code below was used in partenon to convert the greenery FSMs to tensors.  We can repurpose it to convert into a transducer from characters to token names.

In [21]:
def make_matrix_from_fsms(fsms, chars):
    max_states = max([len(fsm.states) for fsm in fsms])
    n_inputs = len(chars)

    m_shape = (len(fsms), max_states, n_inputs, max_states)
    m = np.zeros(m_shape, dtype=np.int8)
    finals = jnp.zeros((len(fsms), max_states))
    for fsm_idx, fsm in enumerate(fsms):
        for final_state in fsm.finals:
            finals = finals.at[fsm_idx, final_state].set(1)
        fsm_states = fsm.states
        rejection_states = [e for e in fsm_states if not fsm.islive(e)]
        for state in fsm_states:
            arcs = fsm.map[state]
            for input_char, next_state in arcs.items():
                if next_state in rejection_states:  # rejection state
                    continue
                for char in input_char.get_chars():
                    input_idx = chars.index(char)
                    m[fsm_idx, state, input_idx, next_state] = 1

    m = jnp.array(m)

    return m, finals, max_states

## Prefix Grammar

In [22]:
len(g.cnf.rules), len(g.cnf.prefix_grammar.trim().rules), len(g.cnf.prefix_grammar.trim().rules)/len(g.cnf.rules)

(302, 1214, 4.0198675496688745)

In [23]:
#len(g.cnf.prefix_grammar.nullaryremove().trim().rules) / len(g.cnf.rules)

In [24]:
#from dyna.util import Graph
#def unary_graph(self):
#    return Graph([(p.body[0], p.head) for p in self if len(p.body) == 1 and self.is_nonterminal(p.body[0])])

G = g.cnf.prefix_grammar.trim()

#print(G.num_rules, len(G.N), len(G.V))
#U = unary_graph(G)
#C = U.condensation()
#print('largest scc:', max(map(len, C.nodes), default=0))

In [25]:
#C

In [26]:
N = G.nullaryremove()    # could be faster with SCC-based prioritization

In [ ]:
N.unaryremove()   # currently, very solve because we dont have SCC-decomposed Lehmann's algorithm

In [ ]:
g.cnf.prefix_grammar.trim().cnf

In [ ]:
g.prefix_weight([t.type for t in tokens])